In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import sys
sys.path.append('./../../designer/script/')
sys.path.append('./../')
sys.path.append('./../../')
from film import TwoMaterialFilm
from spectrum import Spectrum
from design import BaseDesign
import os

from analyze_utils.structure import plot_layer_thickness
from optimizer.adam_d import adam_optimize
import pickle



In [ ]:


def load_designs(file_dir, filter=lambda x: True) -> list[BaseDesign]:
    fname_ls = os.listdir(file_dir)
    designs = []
    for fname in fname_ls:
        if not filter(fname):
            continue
        with open(file_dir + fname, 'rb') as f:
            designs.append(pickle.load(f))
    return designs


dir = './raw_result/back_reflector/SiO2_TiO2-400to1000nm-700nm/'


fig, (ax, ax2, ax3) = plt.subplots(3, 1)
for layer in [41, 91, 191]:

    losses = []
    init_ots = []
    final_layers = []
    final_ots = []
    for design in load_designs(dir, lambda x: f'_layer{layer}_' in x):
        
        design.film.remove_negative_thickness_layer()
        final_layers.append(design.film.get_layer_number())
        losses.append(design.calculate_loss())
        init_ots.append(design.get_init_ot())
        final_ots.append(design.get_current_ot())

    color = cm.get_cmap('cividis')(1 - layer / 200)
    ax.scatter(init_ots, losses, label=f'init {layer} layers', color=color)
    ax2.scatter(init_ots, final_layers, label=f'init {layer} layers', color=color)
    ax3.scatter(init_ots, final_ots, label=f'init {layer} layers', color=color)


ax.set_ylabel('loss')
ax.set_yscale('log')

ax2.set_ylabel('final layer number')
ax2.legend(bbox_to_anchor=(1, 1))

ax3.set_xlabel('init ot / nm')
ax3.set_ylabel('final ot')
# ax3.legend()

fig.tight_layout()


In [ ]:
from analyze_utils.structure import plot_layer_thickness

def plot_deisgn(design):
    design.film.remove_negative_thickness_layer()
    print(f'init layer: {design.init_film.get_layer_number()}')
    print(f'init ot: {design.get_init_ot()}')
    print(f'loss: {design.calculate_loss()}')
    plot_layer_thickness(design.film)

layer = 190
import re
filter = lambda x: f'_layer{layer}_' in x and 5000 > float(re.match('ot(.*)_layer', x).group(1)) > 4900
designs = load_designs(dir, filter)

for design in designs:

    plot_deisgn(design)

    fig, ax = plt.subplots(1, 1)
    wls = np.linspace(400, 700, 1000)
    ax.plot(wls, design.film.get_spec(inc_ang=0., wls=wls).get_R())
    ax.set_title(f'init ot {design.get_init_ot()}')

    fig, ax = plt.subplots(1, 1)
    ax.plot([x['step'] for x in design.training_info], [x['loss'] for x in design.training_info])